In [71]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

In [72]:
def read_csv_data(filename):
    return pd.read_csv(filename)

In [73]:
def create_model(lr):
    inputs = Input(shape=(75,))
    x = Dense(16, activation='relu')(inputs)
    x = Dense(16, activation='relu')(x)
    outputs = Dense(9, activation='softmax')(x)
    
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=Adam(lr=lr), loss=MeanSquaredError())
    
    return model

In [74]:
def split_xy(X):
    y = X['target']
    x = X.drop(columns='target')
    
    return x,y

In [75]:
def create_y(y):
    new_y = []
    for i in y.to_list():
        new_row = [0, 0, 0,
                   0, 0, 0,
                   0, 0, 0]
        
        if i == 'Class_1':
            new_row[0] = 1.0
        elif i == 'Class_2':
            new_row[1] = 1.0
        elif i == 'Class_3':
            new_row[2] = 1.0
        elif i == 'Class_4':
            new_row[3] = 1.0
        elif i == 'Class_5':
            new_row[4] = 1.0
        elif i == 'Class_6':
            new_row[5] = 1.0
        elif i == 'Class_7':
            new_row[6] = 1.0
        elif i == 'Class_8':
            new_row[7] = 1.0
        elif i == 'Class_9':
            new_row[8] = 1.0
        else:
            print("ERROR! TARGET IS OUT OF RANGE!")
            
        new_y.append(new_row)
        
    return pd.DataFrame(new_y)

# TRAIN

In [76]:
# READ TRAIN DATA
df = read_csv_data('train.csv')

In [77]:
# CREATE TF MODEL
tf_model = create_model(lr=0.0001)

In [78]:
# SPLIT X AND Y
X, Y = split_xy(df)
# SPLIT Y TARGETS
y = create_y(Y)
# SPLIT TRAIN TEST
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [79]:
tf_model.fit(X.drop(columns='id').to_numpy(), y.to_numpy(), epochs=100, verbose=1)

Epoch 1/100
6250/6250 [==============================] - 8s 1ms/step - loss: 0.1192
Epoch 2/100
6250/6250 [==============================] - 8s 1ms/step - loss: 0.0932
Epoch 3/100
6250/6250 [==============================] - 8s 1ms/step - loss: 0.0905
Epoch 4/100
6250/6250 [==============================] - 6s 1ms/step - loss: 0.0889
Epoch 5/100
6250/6250 [==============================] - 6s 1ms/step - loss: 0.0879
Epoch 6/100
6250/6250 [==============================] - 6s 1ms/step - loss: 0.0875
Epoch 7/100
6250/6250 [==============================] - 6s 1ms/step - loss: 0.0873
Epoch 8/100
6250/6250 [==============================] - 6s 1ms/step - loss: 0.0871
Epoch 9/100
6250/6250 [==============================] - 6s 1ms/step - loss: 0.0870
Epoch 10/100
6250/6250 [==============================] - 6s 1ms/step - loss: 0.0870
Epoch 11/100
6250/6250 [==============================] - 6s 1ms/step - loss: 0.0868
Epoch 12/100
6250/6250 [==============================] - 6s 1ms/step - lo

# TEST

In [80]:
y_pred = tf_model.predict(X_test.drop(columns='id').to_numpy())

In [81]:
y_pred

array([[0.07046665, 0.08757806, 0.08986833, ..., 0.09971991, 0.26885033,
        0.19392015],
       [0.05174483, 0.11408714, 0.09456849, ..., 0.09652721, 0.28581798,
        0.16788454],
       [0.03583154, 0.10610518, 0.08055083, ..., 0.08072391, 0.30400673,
        0.12970854],
       ...,
       [0.0512664 , 0.05366268, 0.0472685 , ..., 0.10662154, 0.38595203,
        0.13872255],
       [0.06299122, 0.08646193, 0.06604833, ..., 0.09357388, 0.3108606 ,
        0.16342254],
       [0.04223694, 0.158954  , 0.11376303, ..., 0.07096929, 0.23962046,
        0.15009235]], dtype=float32)

In [82]:
r2_score(y_test, y_pred)

0.03846453736312635

# MY PREDICTION

In [83]:
# READ TEST DATA
df = read_csv_data('test.csv')
y_pred = tf_model.predict(df.drop(columns='id').to_numpy())

In [84]:
# CREATE SUBMISSION FILE
id = df['id'].to_list()

# CREATE DICTIONARY FOR DATAFRAME
dataframe_dictionary = {
    'id': id,
    'Class_1': y_pred[:,0],
    'Class_2': y_pred[:,1],
    'Class_3': y_pred[:,2],
    'Class_4': y_pred[:,3],
    'Class_5': y_pred[:,4],
    'Class_6': y_pred[:,5],
    'Class_7': y_pred[:,6],
    'Class_8': y_pred[:,7],
    'Class_9': y_pred[:,8],
}

# CREATE DATAFRAME
submission_df = pd.DataFrame(dataframe_dictionary, index=None)
submission_df.to_csv('submission.csv', index=None)